In [8]:
import pandas as pd
import numpy as np
import json
import requests
import datetime
import twint
import nest_asyncio
nest_asyncio.apply()
import config as cfg
pd.set_option('display.float_format', lambda x: '%.3f' % x) #avoid scientific notation
import time

# EDA

## Coingecko
1. Coins
2. Exchanges
3. Bitcoin price, market_cap, volume
4. ...

In [2]:
base_url = 'https://api.coingecko.com/api/v3'

In [3]:
# fetching list of coins with id and name
url = base_url+'/coins/list'
response = requests.get(url)
coins_list = response.json()
coins_df = pd.DataFrame(coins_list)
#coins_df
print('total listed coins in coingecko: ',len(coins_df))
coins_df[coins_df.symbol.isin(['btc','eth','ada','bnb','sol','doge','cake'])]

total listed coins in coingecko:  7537


id symbol                  name
830            binancecoin    bnb          Binance Coin
834    binance-peg-cardano    ada   Binance-Peg Cardano
835   binance-peg-dogecoin   doge  Binance-Peg Dogecoin
890                bitcoin    btc               Bitcoin
1364               cardano    ada               Cardano
2173              dogecoin   doge              Dogecoin
2501              ethereum    eth              Ethereum
4820     pancakeswap-token   cake           PancakeSwap
5918                solana    sol                Solana

In [4]:
# fetching list of exchanges
exchanges_list = []
for page in range(5):
    url = base_url+'/exchanges?per_page=250&page='+str(page+1)
    response = requests.get(url)
    exchanges_list += response.json()
exchanges_df = pd.DataFrame(exchanges_list)
print('total listed exchanges in coingecko: ',len(exchanges_df))
exchanges_df

total listed exchanges in coingecko:  318


id                name  year_established  \
0             binance             Binance          2017.000   
1                gdax   Coinbase Exchange          2012.000   
2              kraken              Kraken          2011.000   
3                gate             Gate.io               nan   
4            bitfinex            Bitfinex          2014.000   
..                ...                 ...               ...   
313       pantherswap         PantherSwap               nan   
314              idex                Idex          2017.000   
315  binance_dex_mini  Binance DEX (Mini)               nan   
316        delta_spot      Delta Exchange               nan   
317        secretswap          SecretSwap               nan   

                    country  \
0            Cayman Islands   
1             United States   
2             United States   
3                 Hong Kong   
4    British Virgin Islands   
..                      ...   
313                    None   
314                  Panama   
315                    None   
316                    None   
317                    None   

                                           description  \
0                                                        
1                                                        
2                                                        
3                                                        
4                                                        
..                                                 ...   
313                                                      
314                                               None   
315                                                      
316                                                      
317  SecretSwap is a decentralized exchange that pr...   

                                                url  \
0                          https://www.binance.com/   
1                          https://www.coinbase.com   
2                        https://r.kraken.com/Q1m9x   
3                                  https://gate.io/   
4                          https://www.bitfinex.com   
..                                              ...   
313                    https://dex.pantherswap.com/   
314                             https://idex.market   
315                                                   
316  https://www.delta.exchange/app/futures/markets   
317           https://bridge.scrt.network/swap#Swap   

                                                 image has_trading_incentive  \
0    https://assets.coingecko.com/markets/images/52...                 False   
1    https://assets.coingecko.com/markets/images/23...                 False   
2    https://assets.coingecko.com/markets/images/29...                 False   
3    https://assets.coingecko.com/markets/images/60...                 False   
4    https://assets.coingecko.com/markets/images/4/...                 False   
..                                                 ...                   ...   
313  https://assets.coingecko.com/markets/images/67...                 False   
314  https://assets.coingecko.com/markets/images/12...                 False   
315                                  missing_small.png                 False   
316  https://assets.coingecko.com/markets/images/64...                 False   
317  https://assets.coingecko.com/markets/images/66...                 False   

     trust_score  trust_score_rank  trade_volume_24h_btc  \
0         10.000             1.000            853281.502   
1         10.000             2.000            142654.483   
2         10.000             3.000             60090.954   
3         10.000             4.000             33825.966   
4         10.000             5.000             31835.436   
..           ...               ...                   ...   
313          nan           407.000               344.858   
314          nan           412.000                 0.049   
315          nan           414

In [5]:
# fetching btc data (market_chart)
url = base_url+'/coins/bitcoin/market_chart?vs_currency=usd&days=max'
response = requests.get(url)
btc_dict = response.json()

timestamp = [item[0] for item in btc_dict['prices']]
prices = [item[1] for item in btc_dict['prices']]
market_caps = [item[1] for item in btc_dict['market_caps']]
total_volumes = [item[1] for item in btc_dict['total_volumes']]
new_dict = {'timestamp':timestamp,'prices':prices,'market_caps':market_caps,'total_volumes':total_volumes}
btc_df = pd.DataFrame(new_dict)
btc_df

timestamp    prices      market_caps   total_volumes
0     1367107200000   135.300   1500517590.000           0.000
1     1367193600000   141.960   1575032004.000           0.000
2     1367280000000   135.300   1501657493.000           0.000
3     1367366400000   117.000   1298951550.000           0.000
4     1367452800000   103.430   1148667722.000           0.000
...             ...       ...              ...             ...
2945  1621728000000 37520.318 702143968883.525 61178935273.653
2946  1621814400000 34977.528 654558962027.584 84358034875.513
2947  1621900800000 38642.442 723143072419.724 71163197727.335
2948  1621987200000 38210.805 715248689989.895 48724794157.985
2949  1622124216000 39990.713 751302932377.432 45953973886.406

[2950 rows x 4 columns]

## Glassnode (with API key)
1. Actives Address
2. ...

In [3]:
# insert your API key here
API_KEY = cfg.GLASSNODE_API_KEY

# make API request
#url = 'https://api.glassnode.com/v1/metrics/addresses/active_count'
#response = requests.get(url, params={'a': 'BTC', 'api_key': API_KEY})


In [7]:
# convert to pandas dataframe
address_active_df = pd.read_json(response.text, convert_dates=['t'])
#address_active_df.to_csv('address_active_df-glassnode.csv',index=False)
address_active_df

prices                         market_caps  \
0                  [1367107200000, 135.3]         [1367107200000, 1500517590]   
1                 [1367193600000, 141.96]       [1367193600000, 1575032004.0]   
2                  [1367280000000, 135.3]       [1367280000000, 1501657493.0]   
3                  [1367366400000, 117.0]       [1367366400000, 1298951550.0]   
4                 [1367452800000, 103.43]       [1367452800000, 1148667722.0]   
...                                   ...                                 ...   
2945   [1621728000000, 37520.31767547438]  [1621728000000, 702143968883.5249]   
2946  [1621814400000, 34977.527801962424]  [1621814400000, 654558962027.5841]   
2947    [1621900800000, 38642.4422973396]  [1621900800000, 723143072419.7241]   
2948   [1621987200000, 38210.80521759389]  [1621987200000, 715248689989.8947]   
2949  [1622124216000, 39990.713259916476]  [1622124216000, 751302932377.4319]   

                            total_volumes  
0                      [1367107200000, 0]  
1                    [1367193600000, 0.0]  
2                    [1367280000000, 0.0]  
3                    [1367366400000, 0.0]  
4                    [1367452800000, 0.0]  
...                                   ...  
2945  [1621728000000, 61178935273.653305]  
2946   [1621814400000, 84358034875.51308]  
2947    [1621900800000, 71163197727.3351]  
2948  [1621987200000, 48724794157.985054]  
2949   [1622124216000, 45953973886.40648]  

[2950 rows x 3 columns]

## Twitter - Twint
1. Twint: Bitcoin tweets -> too many, overload
2. Twint: Whale Alert -> on going
3. ...

### Search = 'bitcoin'

In [41]:
# scrapping twitter for bitcoin tweets

c = twint.Config()
c.Search = 'bitcoin'
c.Pandas = True
c.Lang = 'en'
c.Limit = 200
#c.Count = True
c

Config(Username=None, User_id=None, Search='bitcoin', Lookup=False, Geo='', Location=False, Near=None, Lang='en', Output=None, Elasticsearch=None, Year=None, Since=None, Until=None, Email=None, Phone=None, Verified=False, Store_csv=False, Store_json=False, Show_hashtags=False, Show_cashtags=False, Limit=200, Count=None, Stats=False, Database=None, To=None, Debug=False, Essid='', Profile=False, Followers=False, Following=False, Favorites=False, TwitterSearch=False, User_full=False, Store_object=False, Store_object_tweets_list=None, Store_object_users_list=None, Store_object_follow_list=None, Pandas_type=None, Pandas=True, Index_tweets='twinttweets', Index_follow='twintgraph', Index_users='twintuser', Retries_count=10, Resume=None, Images=False, Videos=False, Media=False, Replies=False, Pandas_clean=True, Lowercase=True, Pandas_au=True, Proxy_host='', Proxy_port=0, Proxy_type=None, Tor_control_port=9051, Tor_control_password=None, Retweets=False, Query=None, Hide_output=False, Custom_que

In [46]:
twint.run.Search(c)
twbitcoin_df = twint.storage.panda.Tweets_df

1398157446770790401 2021-05-28 08:01:26 +0200 <whale_alert> 🚨 🚨 🚨  39,169,230 #BUSD (39,169,230 USD) transferred from #Binance to unknown wallet   https://t.co/gfXzoPXCwY
1398156795080851462 2021-05-28 07:58:51 +0200 <whale_alert> 🚨 🚨  10,246 #ETH (26,261,313 USD) transferred from unknown wallet to #OKEx   https://t.co/I0jTU46eL1
1398151246415073281 2021-05-28 07:36:48 +0200 <whale_alert> 🚨 🚨 🚨 🚨 🚨  1,350 #BTC (50,331,215 USD) transferred from unknown wallet to #Bitstamp   https://t.co/DV9h7MCzzp
1398145688152727553 2021-05-28 07:14:43 +0200 <whale_alert> 🔥 🔥  19,765,511 #HUSD (19,765,511 USD) burned at HUSD Incinerator   https://t.co/ewBLEJpEWh
1398138163030278144 2021-05-28 06:44:48 +0200 <whale_alert> 🔥  9,888,555 #HUSD (9,888,555 USD) burned at HUSD Incinerator   https://t.co/yrYuRywQha
1398113709583654917 2021-05-28 05:07:38 +0200 <whale_alert> 🚨 🚨 🚨  1,138,262 #UNI (32,879,799 USD) transferred from unknown wallet to unknown wallet   https://t.co/9nUQM2Su4D
1398112211634208770 202

1397863792596426756 2021-05-27 12:34:33 +0200 <whale_alert> 💵 💵  19,565,521 #USDC (19,565,521 USD) minted at USDC Treasury   https://t.co/dhleCzeAfm
1397862878514982913 2021-05-27 12:30:55 +0200 <whale_alert> 🚨 🚨 🚨  39,547,610 #BUSD (39,547,610 USD) transferred from unknown wallet to #Binance   https://t.co/S0h4QrTK1F
1397861646987677699 2021-05-27 12:26:02 +0200 <whale_alert> 🔥 🔥 🔥 🔥  32,655,254 #USDC (32,655,254 USD) burned at USDC Treasury   https://t.co/QcGUh1Rsle
1397860294425260033 2021-05-27 12:20:39 +0200 <whale_alert> 💵 💵  19,565,519 #USDC (19,565,519 USD) minted at USDC Treasury   https://t.co/UekX2npYds
1397858968345661442 2021-05-27 12:15:23 +0200 <whale_alert> 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨  70,000 #ETH (195,335,518 USD) transferred from unknown wallet to unknown wallet   https://t.co/XlH0kS9MZl
1397858086233247744 2021-05-27 12:11:53 +0200 <whale_alert> 665,623 #OMG (5,230,193 USD) transferred from #Binance to unknown wallet   https://t.co/omEfIQdz0h
1397852684502323203 2021-05-27 1

In [47]:
len(twbitcoin_df)

200

In [48]:
twbitcoin_df

id      conversation_id        created_at  \
0    1398157446770790401  1398157446770790401 1622181686000.000   
1    1398156795080851462  1398156795080851462 1622181531000.000   
2    1398151246415073281  1398151246415073281 1622180208000.000   
3    1398145688152727553  1398145688152727553 1622178883000.000   
4    1398138163030278144  1398138163030278144 1622177088000.000   
..                   ...                  ...               ...   
195  1397607324290850817  1397607324290850817 1622050527000.000   
196  1397607322067902469  1397607322067902469 1622050526000.000   
197  1397606439896731652  1397606439896731652 1622050316000.000   
198  1397603292146974720  1397603292146974720 1622049565000.000   
199  1397603289341046785  1397603289341046785 1622049565000.000   

                    date timezone place  \
0    2021-05-28 08:01:26    +0200         
1    2021-05-28 07:58:51    +0200         
2    2021-05-28 07:36:48    +0200         
3    2021-05-28 07:14:43    +0200         
4    2021-05-28 06:44:48    +0200         
..                   ...      ...   ...   
195  2021-05-26 19:35:27    +0200         
196  2021-05-26 19:35:26    +0200         
197  2021-05-26 19:31:56    +0200         
198  2021-05-26 19:19:25    +0200         
199  2021-05-26 19:19:25    +0200         

                                                 tweet language  \
0    🚨 🚨 🚨  39,169,230 #BUSD (39,169,230 USD) trans...       en   
1    🚨 🚨  10,246 #ETH (26,261,313 USD) transferred ...       en   
2    🚨 🚨 🚨 🚨 🚨  1,350 #BTC (50,331,215 USD) transfe...       en   
3    🔥 🔥  19,765,511 #HUSD (19,765,511 USD) burned ...       sv   
4    🔥  9,888,555 #HUSD (9,888,555 USD) burned at H...       sv   
..                                                 ...      ...   
195  🚨 🚨 🚨 🚨  1,126 #BTC (43,238,194 USD) transferr...       en   
196  🚨 🚨 🚨 🚨  1,200 #BTC (46,080,005 USD) transferr...       en   
197  🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨  101,999,990 #BUSD (101,99...       en   
198  🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨  101,999,990 #BUSD (101,99...       en   
199  💵  6,820,719 #USDC (6,820,719 USD) minted at U...       en   

                     hashtags cashtags  ...  geo source user_rt_id user_rt  \
0             [busd, binance]       []  ...                                  
1                 [eth, okex]       []  ...                                  
2             [btc, bitstamp]       []  ...                                  
3                      [husd]       []  ...                                  
4                      [husd]       []  ...                                  
..                        ...      ...  ...  ...    ...        ...     ...   
195  [btc, binance, bitstamp]       []  ...                                  
196             [btc, kraken]       []  ...                                  
197           [busd, binance]       []  ...                                  
198           [busd, binance]       []  ...                                  
199                    [usdc]       []  ...                                  

     retweet_id reply_to retweet_date translate trans_src  trans_dest  
0                     []                                               
1                     []                                               
2                     []                                               
3                     []                                               
4                     []                                               
..          ...      ...          ...       ...       ...         ...  
195                   []                                               
196                   []                                               
197                   []                                               
198                   []                                               
199                   []                                               

[200 rows x 38 columns]

### Username = 'whale_alert'

#### Creating first base of tweets

In [30]:
date_from_ls = []
date_until_ls = []
n_weeks = 22 #10 #22

for week in range(n_weeks):
#    print(datetime.datetime.today().date() - datetime.timedelta(days=7*week))
    yesterday = datetime.datetime.today().date() - datetime.timedelta(days=1)
    date_from = yesterday - datetime.timedelta(days=7*(week+1))
    date_until = yesterday - datetime.timedelta(days=7*(week))
    date_from_ls.append(str(date_from)) 
    date_until_ls.append(str(date_until))
#    print('\tfrom: {} \t until: {}'.format(date_from,date_until))
print('date_from: ', date_from_ls)
print('date_until: ', date_until_ls)
dates_np = np.array([date_from_ls,date_until_ls])

date_from:  ['2021-05-22', '2021-05-15', '2021-05-08', '2021-05-01', '2021-04-24', '2021-04-17', '2021-04-10', '2021-04-03', '2021-03-27', '2021-03-20', '2021-03-13', '2021-03-06', '2021-02-27', '2021-02-20', '2021-02-13', '2021-02-06', '2021-01-30', '2021-01-23', '2021-01-16', '2021-01-09', '2021-01-02', '2020-12-26']
date_until:  ['2021-05-29', '2021-05-22', '2021-05-15', '2021-05-08', '2021-05-01', '2021-04-24', '2021-04-17', '2021-04-10', '2021-04-03', '2021-03-27', '2021-03-20', '2021-03-13', '2021-03-06', '2021-02-27', '2021-02-20', '2021-02-13', '2021-02-06', '2021-01-30', '2021-01-23', '2021-01-16', '2021-01-09', '2021-01-02']


In [31]:
n_weeks = dates_np.shape[1]
for i in range(n_weeks):
    print('{}) {} - {}'.format(i+1, dates_np[0,i], dates_np[1,i]))

1) 2021-05-22 - 2021-05-29
2) 2021-05-15 - 2021-05-22
3) 2021-05-08 - 2021-05-15
4) 2021-05-01 - 2021-05-08
5) 2021-04-24 - 2021-05-01
6) 2021-04-17 - 2021-04-24
7) 2021-04-10 - 2021-04-17
8) 2021-04-03 - 2021-04-10
9) 2021-03-27 - 2021-04-03
10) 2021-03-20 - 2021-03-27
11) 2021-03-13 - 2021-03-20
12) 2021-03-06 - 2021-03-13
13) 2021-02-27 - 2021-03-06
14) 2021-02-20 - 2021-02-27
15) 2021-02-13 - 2021-02-20
16) 2021-02-06 - 2021-02-13
17) 2021-01-30 - 2021-02-06
18) 2021-01-23 - 2021-01-30
19) 2021-01-16 - 2021-01-23
20) 2021-01-09 - 2021-01-16
21) 2021-01-02 - 2021-01-09
22) 2020-12-26 - 2021-01-02


In [44]:
dates_np[:,11]

array(['2021-03-06', '2021-03-13'], dtype='<U10')

In [45]:
for i in range(11,n_weeks):
    print(i)

11
12
13
14
15
16
17
18
19
20
21


In [46]:
# scrapping tweets from 'whale_alert'
twwhale_df = pd.DataFrame()
n_weeks = dates_np.shape[1]
for i in range(11, n_weeks):
        
    c = twint.Config()
    c.Username = 'whale_alert'
    c.Pandas = True
    #c.Lang = 'en'
    #c.Limit = 200
    #c.Count = True
    c.Since = dates_np[0,i]
    c.Until = dates_np[1,i]
    twint.run.Search(c)
    twwhale_df = twwhale_df.append(twint.storage.panda.Tweets_df)
    time.sleep(2)

1370518786047102976 2021-03-13 00:35:16 +0200 <whale_alert> 27,000 #BNB (7,146,027 USD) transferred from #Binance to unknown wallet   https://t.co/g2FnMrFqtM
1370516817689653250 2021-03-13 00:27:26 +0200 <whale_alert> 21,000 #BNB (5,530,644 USD) transferred from #Binance to unknown wallet   https://t.co/Bm4q6e4Xbs
1370505301703729159 2021-03-12 23:41:41 +0200 <whale_alert> 27,000 #BNB (7,132,653 USD) transferred from #Binance to unknown wallet   https://t.co/mGcTfes8LM
1370503098561662977 2021-03-12 23:32:56 +0200 <whale_alert> 2,396,774 #REN (2,774,563 USD) transferred from #FTX to unknown wallet   https://t.co/vbxNGtn5cF
1370503095571070980 2021-03-12 23:32:55 +0200 <whale_alert> 2,489,606 #REN (2,882,028 USD) transferred from #FTX to #Binance   https://t.co/U1Tobo0tPa
1370500314315550728 2021-03-12 23:21:52 +0200 <whale_alert> 25,000 #BNB (6,571,645 USD) transferred from #Binance to unknown wallet   https://t.co/2WgZz40keZ
1370500081271631872 2021-03-12 23:20:56 +0200 <whale_alert> 

1370318102068658176 2021-03-12 11:17:49 +0200 <whale_alert> 🚨 🚨  400 #BTC (22,394,801 USD) transferred from #Xapo to #Binance   https://t.co/qpTTUZKg8n
1370318100621578242 2021-03-12 11:17:49 +0200 <whale_alert> 🚨 🚨 🚨 🚨 🚨 🚨  1,180 #BTC (66,064,662 USD) transferred from #Xapo to #Binance   https://t.co/mWulMroI73
1370318099266863107 2021-03-12 11:17:48 +0200 <whale_alert> 🚨 🚨 🚨 🚨  800 #BTC (44,789,602 USD) transferred from unknown wallet to CryptoCom   https://t.co/Or5KG3Zt3K
1370318097324867586 2021-03-12 11:17:48 +0200 <whale_alert> 🚨 🚨  500 #BTC (27,993,501 USD) transferred from unknown wallet to CryptoCom   https://t.co/alFWwt5fOC
1370315646446272512 2021-03-12 11:08:04 +0200 <whale_alert> 100,000,000 #FUN (3,086,995 USD) transferred from #Binance to unknown wallet   https://t.co/FlBDvgq6SQ
1370312001436680192 2021-03-12 10:53:34 +0200 <whale_alert> 🚨  11,835,406 #USDC (11,835,406 USD) transferred from unknown wallet to #FTX   https://t.co/oZeDUZapRA
1370308882694815747 2021-03-12 1

1370150725179875328 2021-03-12 00:12:43 +0200 <whale_alert> 🚨 🚨 🚨 🚨  23,507 #ETH (43,337,978 USD) transferred from unknown wallet to unknown wallet   https://t.co/RIgHgRRxd8
1370149830463528962 2021-03-12 00:09:10 +0200 <whale_alert> 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨  2,901 #BTC (167,322,971 USD) transferred from unknown wallet to unknown wallet   https://t.co/Aup8FYFLmW
1370144787731451906 2021-03-11 23:49:08 +0200 <whale_alert> 🚨  8,000 #ETH (14,607,267 USD) transferred from #CryptoCom to unknown wallet   https://t.co/dPdFEIffi8
1370144125220225034 2021-03-11 23:46:30 +0200 <whale_alert> 🚨  15,000,000 #USDC (15,000,000 USD) transferred from unknown wallet to #Bitfinex   https://t.co/TV8PXgMZcC
1370140404612538373 2021-03-11 23:31:43 +0200 <whale_alert> 🚨  11,598,863 #USDT (11,584,709 USD) transferred from #Okex to #Binance   https://t.co/6gbqdIGkIx
1370140402637008897 2021-03-11 23:31:42 +0200 <whale_alert> 🚨  11,623,189 #USDT (11,609,005 USD) transferred from #Okex to #Binance   https://t.co/T3Fo

1369951975220084736 2021-03-11 11:02:58 +0200 <whale_alert> 🚨 🚨 🚨  600 #BTC (32,804,569 USD) transferred from #Binance to #Coinbase  Tx:  https://t.co/o9moZnHiOX
1369950512657874945 2021-03-11 10:57:09 +0200 <whale_alert> 🚨 🚨  1,000,000 #LINK (28,885,802 USD) transferred from #CryptoCom to unknown wallet  Tx:  https://t.co/ITwS0iznrM
1369950329828036612 2021-03-11 10:56:25 +0200 <whale_alert> 🚨  7,500 #ETH (13,065,891 USD) transferred from unknown wallet to #Binance  Tx:  https://t.co/0iBGWn1zNn
1369950275398733826 2021-03-11 10:56:12 +0200 <whale_alert> 100,000,000 #FUN (3,223,993 USD) transferred from #Binance to unknown wallet  Tx:  https://t.co/6JT6K7Ip1x
1369946600148062208 2021-03-11 10:41:36 +0200 <whale_alert> 🚨 🚨 🚨  30,712,972 #USDT (30,716,029 USD) transferred from #FTX to #Binance  Tx:  https://t.co/S8200kk5w4
1369946598042664962 2021-03-11 10:41:36 +0200 <whale_alert> 🚨 🚨 🚨  30,139,882 #USDT (30,142,882 USD) transferred from #FTX to #Binance  Tx:  https://t.co/X7KpEz3QPp
13

1369742926314635271 2021-03-10 21:12:16 +0200 <whale_alert> 🚨 🚨  25,000,000 #USDT (25,022,163 USD) transferred from unknown wallet to unknown wallet  Tx:  https://t.co/W1RPxRpdoo
1369740409405071360 2021-03-10 21:02:16 +0200 <whale_alert> 🚨 🚨  10,995 #ETH (20,299,275 USD) transferred from unknown wallet to unknown wallet  Tx:  https://t.co/gpYjdOIel9
1369739929551577093 2021-03-10 21:00:22 +0200 <whale_alert> 🚨 🚨 🚨  30,000,000 #USDC (30,000,000 USD) transferred from unknown wallet to unknown wallet  Tx:  https://t.co/BcExzTYe8N
1369738717829693447 2021-03-10 20:55:33 +0200 <whale_alert> 🚨 🚨 🚨 🚨  24,500 #ETH (45,456,344 USD) transferred from unknown wallet to unknown wallet  Tx:  https://t.co/mGrxj9VQYC
1369738328518627328 2021-03-10 20:54:00 +0200 <whale_alert> 🚨 🚨 🚨 🚨  24,500 #ETH (45,215,218 USD) transferred from unknown wallet to unknown wallet  Tx:  https://t.co/CkfL8Zx4Cn
1369737159914184708 2021-03-10 20:49:22 +0200 <whale_alert> 💵 💵  13,680,921 #USDC (13,680,921 USD) minted at U

...

1344789133617717253 2021-01-01 00:34:49 +0200 <whale_alert> 🚨  678 #BTC (19,612,260 USD) transferred from #Bitstamp to unknown wallet  Tx:  https://t.co/pKffiX83Wq
1344789132250411008 2021-01-01 00:34:48 +0200 <whale_alert> 🚨  351 #BTC (10,153,249 USD) transferred from #Coinbase to unknown wallet  Tx:  https://t.co/MTaotUFTyl
1344789130245505027 2021-01-01 00:34:48 +0200 <whale_alert> 🚨  387 #BTC (11,206,179 USD) transferred from #Coinbase to #Bitstamp  Tx:  https://t.co/TiFrqgFQHn
1344787332868820992 2021-01-01 00:27:39 +0200 <whale_alert> 🔥  9,106,481 #USDC (9,106,481 USD) burned at USDC Incinerator  Tx:  https://t.co/Gq0jjT8HD7
1344783368135864329 2021-01-01 00:11:54 +0200 <whale_alert> 🚨 🚨 🚨 🚨  1,619 #BTC (46,713,623 USD) transferred from #Bitstamp to unknown wallet  Tx:  https://t.co/hWH0OzAT4R
1344776955921641475 2020-12-31 23:46:25 +0200 <whale_alert> 🚨  656 #BTC (19,028,966 USD) transferred from #Bitstamp to unknown wallet  Tx:  https://t.co/T7tjkVTHVj
1344766079621603331 2020-

1344533255484289024 2020-12-31 07:38:02 +0200 <whale_alert> 🚨 🚨 🚨  41,800 #ETH (31,166,529 USD) transferred from unknown wallet to unknown wallet  Tx:  https://t.co/t0vpQDPHB0
1344533249100541953 2020-12-31 07:38:01 +0200 <whale_alert> 🚨  421 #BTC (12,257,483 USD) transferred from unknown wallet to Bitbank  Tx:  https://t.co/1TWgFurHho
1344528355044032512 2020-12-31 07:18:34 +0200 <whale_alert> 💵  5,434,526 #USDC (5,434,526 USD) minted at USDC Treasury  Tx:  https://t.co/JRt51URl4E
1344525590397902848 2020-12-31 07:07:35 +0200 <whale_alert> 260,000 #LINK (2,910,636 USD) transferred from unknown wallet to unknown wallet  Tx:  https://t.co/474HxAAyHt
1344518015262617600 2020-12-31 06:37:29 +0200 <whale_alert> 🚨  404 #BTC (11,652,954 USD) transferred from #Bitstamp to unknown wallet  Tx:  https://t.co/w7PCfjSBsk
1344518013228363778 2020-12-31 06:37:28 +0200 <whale_alert> 🚨  424 #BTC (12,222,864 USD) transferred from Bitbank to unknown wallet  Tx:  https://t.co/IJMeUrmx65
13445157287000064

1344150703263141889 2020-12-30 06:17:55 +0200 <whale_alert> 🚨 🚨  986 #BTC (27,632,519 USD) transferred from #Coinbase to unknown wallet  Tx:  https://t.co/eMLZhtEGFR
1344146938556510208 2020-12-30 06:02:57 +0200 <whale_alert> 🚨  13,029,977 #BUSD (13,029,977 USD) transferred from #Huobi to #FTX  Tx:  https://t.co/HU5vi16qaU
1344146855538663424 2020-12-30 06:02:38 +0200 <whale_alert> 🚨  9,980,000 #PAX (10,012,487 USD) transferred from #Binance to unknown wallet  Tx:  https://t.co/MaTEeyHVTj
1344143404423409666 2020-12-30 05:48:55 +0200 <whale_alert> 🚨  11,581,785 #BUSD (11,581,785 USD) transferred from PAX Treasury to unknown wallet  Tx:  https://t.co/a81TjuHrjq
1344143154644197376 2020-12-30 05:47:55 +0200 <whale_alert> 💵 💵 💵  24,611,761 #BUSD (24,611,761 USD) minted at PAX Treasury  Tx:  https://t.co/JHKn8AhIqG
1344140646161018880 2020-12-30 05:37:57 +0200 <whale_alert> 🚨  11,172,629 #BUSD (11,172,629 USD) transferred from #Huobi to #FTX  Tx:  https://t.co/NC18oLMx4F
134413540633444761

1343745311076970496 2020-12-29 03:27:02 +0200 <whale_alert> 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨  3,955 #BTC (105,567,312 USD) transferred from unknown wallet to unknown wallet  Tx:  https://t.co/XQpUjmILuY
1343738612169510912 2020-12-29 03:00:25 +0200 <whale_alert> 689,214 #ATOM (3,735,771 USD) transferred from unknown wallet to unknown wallet  Tx:  https://t.co/MANYm34qiQ
1343733448905814025 2020-12-29 02:39:54 +0200 <whale_alert> 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨 🚨  4,695 #BTC (125,714,798 USD) transferred from unknown wallet to unknown wallet  Tx:  https://t.co/Suw50mPSTb
1343720624552865794 2020-12-29 01:48:56 +0200 <whale_alert> 🚨  501 #BTC (13,392,318 USD) transferred from #Coinbase to unknown wallet  Tx:  https://t.co/79HzwRmzxv
1343718055977897985 2020-12-29 01:38:44 +0200 <whale_alert> 🚨  460 #BTC (12,385,609 USD) transferred from #Xapo to unknown wallet  Tx:  https://t.co/5NOHtLj74X
1343712792227016706 2020-12-29 01:17:49 +0200 <whale_alert> 110,050 #BNB (3,945,304 USD) transferred from unknown wallet to un

1343254005644992512 2020-12-27 18:54:46 +0200 <whale_alert> 🔥  8,936,306 #BUSD (8,936,306 USD) burned at PAX Treasury  Tx:  https://t.co/E2mal0HCCo
1343244280110993411 2020-12-27 18:16:07 +0200 <whale_alert> 500,000 #XTZ (1,028,271 USD) transferred from unknown wallet to #Coinbase  Tx:  https://t.co/wDem81XVDm
1343242851614937088 2020-12-27 18:10:26 +0200 <whale_alert> 🚨  500 #BTC (13,372,578 USD) transferred from #Coinbase to unknown wallet  Tx:  https://t.co/aIYaDtsNtN
1343237793481453569 2020-12-27 17:50:20 +0200 <whale_alert> 9,000,000 #XLM (1,342,252 USD) transferred from #Bittrex to unknown wallet  Tx:  https://t.co/n9VKfEtWgK
1343234874413604865 2020-12-27 17:38:44 +0200 <whale_alert> 500,000 #XTZ (1,034,599 USD) transferred from #Kraken to unknown wallet  Tx:  https://t.co/nI0wLSSDHG
1343214930418393088 2020-12-27 16:19:29 +0200 <whale_alert> 500,000 #XTZ (1,041,321 USD) transferred from unknown wallet to #Binance  Tx:  https://t.co/Pk78RP2kRq
1343209882703581184 2020-12-27 15:

In [47]:
count_dup = twwhale_df.id.duplicated().sum()
if count_dup == 0:
    print('Success: No duplicates retreived.')
else:
    print('Check for duplicates.')

Success: No duplicates retreived.


In [50]:
old_twwhale_df = pd.read_csv('twwhale_df.csv')
print('old_twwhale_df.shape: \t', old_twwhale_df.shape)
twwhale_df = twwhale_df.append(old_twwhale_df).reset_index(drop=True).drop_duplicates(subset='id')
print('twwhale_df.shape: \t', twwhale_df.shape)
twwhale_df.to_csv('twwhale_df.csv', index=False)
print('Saved successfully. \nAdded: {}.'.format(len(twwhale_df)-len(old_twwhale_df)))

old_twwhale_df.shape: 	 (31327, 38)
twwhale_df.shape: 	 (31327, 38)
Saved successfully. 
Added: 0.


## Whale Alert API

In [26]:
# list of connected blockchains
# https://docs.whale-alert.io/?_ga=2.81657559.1136161557.1622185089-376812387.1621839411#status
url = 'https://api.whale-alert.io/v1/status?api_key={}'.format(cfg.WHALE_ALERT_API_KEY)
response = requests.get(url)
pd.DataFrame(response.json()['blockchains'])

name                                            symbols     status
0   binancechain                                              [bnb]  connected
1           icon                                              [icx]  connected
2       ethereum  [uma, wic, hex, hot, tnb, rep, mxm, nexo, etho...  connected
3          steem                                            [steem]  connected
4           tron                              [btt, trx, usdt, bnb]  connected
5          tezos                                      [amount, xtz]  connected
6         cosmos                                             [atom]  connected
7        bitcoin                                  [usdt, eurt, btc]  connected
8        unknown                                                 []  connected
9            eos                                   [eos, usdt, leo]  connected
10          hive                                           [hive, ]  connected
11           neo                                         [neo, gas]  connected
12        ripple                                            [, xrp]  connected
13       stellar                           [slt, xlm, repo, mobi, ]  connected
14        liquid                                             [usdt]  connected

In [63]:
# fetching transactions
# https://docs.whale-alert.io/?_ga=2.81657559.1136161557.1622185089-376812387.1621839411#transactions
start = str(int(datetime.datetime.now().timestamp()-30))

url = 'https://api.whale-alert.io/v1/transactions?api_key={}&min_value=500000&start={}'.format(cfg.WHALE_ALERT_API_KEY,start)
response = requests.get(url)
#pd.DataFrame(response.json()['blockchains'])

In [64]:
response.json()

{'result': 'success', 'cursor': '0-0-0', 'count': 0}

## Kaggle - Bitcoin Tweets
https://www.kaggle.com/alaix14/bitcoin-tweets-20160101-to-20190329 \
Tweets from 2016-01-01 to 2019-03-29 containing 'Bitcoin' or 'BTC' \
Tools used Twint and Tweepy.
File too big to upload in GitHub, only available locally (4GB).

In [38]:
old_btc_tweets = pd.read_csv('../tweets.csv', sep=';')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0,1,2,3,4,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [39]:
len(old_btc_tweets)

20165013

In [40]:
old_btc_tweets

id             user              fullname  url  \
0         1132977055300300800     KamdemAbdiel         Abdiel kamdem  NaN   
1         1132977073402736640        bitcointe             Bitcointe  NaN   
2         1132977023893139456        3eyedbran   Bran - 3 Eyed Raven  NaN   
3         1132977089089556481    DetroitCrypto           J. Scardina  NaN   
4         1132977092340191232     mmursaleen72    Muhammad Mursaleen  NaN   
...                       ...              ...                   ...  ...   
20165008  1198262135580741633    JacobCanfield        Jacob Canfield  NaN   
20165009  1198266433941233664          Vizique               Vizique  NaN   
20165010  1198266441293860864         torusJKL  Gal Buki ($torusJKL)  NaN   
20165011  1198266442673733633  Adekunl95628158       Adekunle Daniel  NaN   
20165012  1198266228583882752       HaraldoXRP                 Harry  NaN   

                       timestamp  replies  likes  retweets  \
0         2019-05-27 11:49:14+00    0.000  0.000     0.000   
1         2019-05-27 11:49:18+00    0.000  0.000     0.000   
2         2019-05-27 11:49:06+00    0.000  2.000     1.000   
3         2019-05-27 11:49:22+00    0.000  0.000     0.000   
4         2019-05-27 11:49:23+00    0.000  0.000     0.000   
...                          ...      ...    ...       ...   
20165008  2019-11-23 15:28:50+00    2.000 16.000     3.000   
20165009  2019-11-23 15:45:55+00    0.000  0.000     0.000   
20165010  2019-11-23 15:45:56+00    0.000  0.000     0.000   
20165011  2019-11-23 15:45:57+00    0.000  0.000     0.000   
20165012  2019-11-23 15:45:06+00    0.000  2.000     1.000   

                                                       text  
0         È appena uscito un nuovo video! LES CRYPTOMONN...  
1         Cardano: Digitize Currencies; EOS https://t.co...  
2         Another Test tweet that wasn't caught in the s...  
3         Current Crypto Prices! \n\nBTC: $8721.99 USD\n...  
4         Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...  
...                                                     ...  
20165008  Happy #FibonacciDay \n\nA while back I created...  
20165009  Bitcoin Suisse Certificates :) https://t.co/nd...  
20165010  Register now for the early access of the Codug...  
20165011  @btc \n@btc \nDo you know that BTC Baskets isn...  
20165012  $BTC - an update on the longer term view for B...  

[20165013 rows x 9 columns]